## CBoW ಮಾದರಿಯನ್ನು ತರಬೇತುಗೊಳಿಸುವುದು

ಈ ನೋಟ್ಬುಕ್ [AI for Beginners Curriculum](http://aka.ms/ai-beginners) ಭಾಗವಾಗಿದೆ

ಈ ಉದಾಹರಣೆಯಲ್ಲಿ, ನಾವು ನಮ್ಮದೇ Word2Vec ಎम्बೆಡ್ಡಿಂಗ್ ಸ್ಪೇಸ್ ಪಡೆಯಲು CBoW ಭಾಷಾ ಮಾದರಿಯನ್ನು ತರಬೇತುಗೊಳಿಸುವುದನ್ನು ನೋಡೋಣ. ನಾವು AG News ಡೇಟಾಸೆಟ್ ಅನ್ನು ಪಠ್ಯದ ಮೂಲವಾಗಿ ಬಳಸುತ್ತೇವೆ.


In [ ]:
import torch
import torchtext
import os
import collections
import builtins
import random
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ಮೊದಲು ನಮ್ಮ ಡೇಟಾಸೆಟ್ ಅನ್ನು ಲೋಡ್ ಮಾಡಿ ಟೋಕನೈಜರ್ ಮತ್ತು ಶಬ್ದಕೋಶವನ್ನು ವ್ಯಾಖ್ಯಾನಿಸೋಣ. ಗಣನೆಗಳನ್ನು ಸ್ವಲ್ಪ ಮಿತಿಗೊಳಿಸಲು ನಾವು `vocab_size` ಅನ್ನು 5000ಕ್ಕೆ ಹೊಂದಿಸುವೆವು.


In [ ]:
def load_dataset(ngrams = 1, min_freq = 1, vocab_size = 5000 , lines_cnt = 500):
    tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
    print("Loading dataset...")
    test_dataset, train_dataset  = torchtext.datasets.AG_NEWS(root='./data')
    train_dataset = list(train_dataset)
    test_dataset = list(test_dataset)
    classes = ['World', 'Sports', 'Business', 'Sci/Tech']
    print('Building vocab...')
    counter = collections.Counter()
    for i, (_, line) in enumerate(train_dataset):
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
        if i == lines_cnt:
            break
    vocab = torchtext.vocab.Vocab(collections.Counter(dict(counter.most_common(vocab_size))), min_freq=min_freq)
    return train_dataset, test_dataset, classes, vocab, tokenizer

In [ ]:
train_dataset, test_dataset, _, vocab, tokenizer = load_dataset()

Loading dataset...
Building vocab...


In [ ]:
def encode(x, vocabulary, tokenizer = tokenizer):
    return [vocabulary[s] for s in tokenizer(x)]

## CBoW ಮಾದರಿ

CBoW ಒಂದು ಪದವನ್ನು ಅದರ $2N$ ಹತ್ತಿರದ ಪದಗಳ ಆಧಾರದ ಮೇಲೆ ಊಹಿಸಲು ಕಲಿಯುತ್ತದೆ. ಉದಾಹರಣೆಗೆ, $N=1$ ಆಗಿದ್ದಾಗ, *I like to train networks* ಎಂಬ ವಾಕ್ಯದಿಂದ ನಾವು ಕೆಳಗಿನ ಜೋಡಿಗಳನ್ನು ಪಡೆಯುತ್ತೇವೆ: (like,I), (I, like), (to, like), (like,to), (train,to), (to, train), (networks, train), (train,networks). ಇಲ್ಲಿ, ಮೊದಲ ಪದವು ಇನ್‌ಪುಟ್ ಆಗಿ ಬಳಸುವ ಹತ್ತಿರದ ಪದವಾಗಿದ್ದು, ಎರಡನೇ ಪದವು ನಾವು ಊಹಿಸುವ ಪದವಾಗಿದೆ.

ಮುಂದಿನ ಪದವನ್ನು ಊಹಿಸಲು ನೆಟ್‌ವರ್ಕ್ ನಿರ್ಮಿಸಲು, ಹತ್ತಿರದ ಪದವನ್ನು ಇನ್‌ಪುಟ್ ಆಗಿ ನೀಡಬೇಕು ಮತ್ತು ಪದದ ಸಂಖ್ಯೆಯನ್ನು ಔಟ್‌ಪುಟ್ ಆಗಿ ಪಡೆಯಬೇಕು. CBoW ನೆಟ್‌ವರ್ಕ್‌ನ ವಾಸ್ತುಶಿಲ್ಪ ಹೀಗಿದೆ:

* ಇನ್‌ಪುಟ್ ಪದವನ್ನು embedding ಲೇಯರ್ ಮೂಲಕ ಕಳುಹಿಸಲಾಗುತ್ತದೆ. ಈ embedding ಲೇಯರ್ ನಮ್ಮ Word2Vec embedding ಆಗಿದ್ದು, ಅದನ್ನು `embedder` ಎಂಬ ವ್ಯತ್ಯಾಸವಾಗಿ ಪ್ರತ್ಯೇಕವಾಗಿ ವ್ಯಾಖ್ಯಾನಿಸುವೆವು. ಈ ಉದಾಹರಣೆಯಲ್ಲಿ embedding ಗಾತ್ರ = 30 ಅನ್ನು ಬಳಸುತ್ತೇವೆ, ಆದರೂ ನೀವು ಹೆಚ್ಚಿನ ಆಯಾಮಗಳೊಂದಿಗೆ ಪ್ರಯೋಗ ಮಾಡಬಹುದು (ನಿಜವಾದ word2vec ನಲ್ಲಿ 300 ಇದೆ)
* embedding ವೆಕ್ಟರ್ ನಂತರ ಔಟ್‌ಪುಟ್ ಪದವನ್ನು ಊಹಿಸುವ ಲೀನಿಯರ್ ಲೇಯರ್‌ಗೆ ಕಳುಹಿಸಲಾಗುತ್ತದೆ. ಆದ್ದರಿಂದ ಇದರಲ್ಲಿ `vocab_size` ನ್ಯೂರಾನ್ಗಳು ಇರುತ್ತವೆ.

ಔಟ್‌ಪುಟ್‌ಗೆ, ನಾವು `CrossEntropyLoss` ಅನ್ನು ನಷ್ಟ ಕಾರ್ಯವಾಗಿ ಬಳಸಿದರೆ, ನಾವು ನಿರೀಕ್ಷಿತ ಫಲಿತಾಂಶಗಳಾಗಿ ಪದದ ಸಂಖ್ಯೆಗಳನ್ನಷ್ಟೇ ಒದಗಿಸಬೇಕು, one-hot ಎನ್‌ಕೋಡಿಂಗ್ ಇಲ್ಲದೆ.


In [ ]:
vocab_size = len(vocab)

embedder = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = 30)
model = torch.nn.Sequential(
    embedder,
    torch.nn.Linear(in_features = 30, out_features = vocab_size),
)

print(model)

Sequential(
  (0): Embedding(5002, 30)
  (1): Linear(in_features=30, out_features=5002, bias=True)
)


## ತರಬೇತಿ ಡೇಟಾ ಸಿದ್ಧತೆ

ಈಗ ನಾವು ಪಠ್ಯದಿಂದ CBoW ಪದ ಜೋಡಿಗಳನ್ನು ಲೆಕ್ಕಹಾಕುವ ಮುಖ್ಯ ಕಾರ್ಯವನ್ನು ಪ್ರೋಗ್ರಾಮ್ ಮಾಡೋಣ. ಈ ಕಾರ್ಯವು ವಿಂಡೋ ಗಾತ್ರವನ್ನು ನಿರ್ದಿಷ್ಟಪಡಿಸಲು ಅವಕಾಶ ನೀಡುತ್ತದೆ ಮತ್ತು ಇನ್‌ಪುಟ್ ಮತ್ತು ಔಟ್‌ಪುಟ್ ಪದಗಳ ಜೋಡಿಗಳ ಸೆಟ್ ಅನ್ನು ಹಿಂತಿರುಗಿಸುತ್ತದೆ. ಈ ಕಾರ್ಯವನ್ನು ಪದಗಳ ಮೇಲೆ ಮಾತ್ರವಲ್ಲದೆ ವೆಕ್ಟರ್‌ಗಳು/ಟೆನ್ಸರ್‌ಗಳ ಮೇಲೆ ಕೂಡ ಬಳಸಬಹುದು - ಇದು ಪಠ್ಯವನ್ನು ಎನ್‌ಕೋಡ್ ಮಾಡಿ ನಂತರ `to_cbow` ಕಾರ್ಯಕ್ಕೆ ಪಾಸ್ ಮಾಡಲು ಸಹಾಯ ಮಾಡುತ್ತದೆ.


In [ ]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(encode('I like to train networks', vocab)))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[232, 172], [5, 172], [172, 232], [5, 232], [0, 232], [172, 5], [232, 5], [0, 5], [1202, 5], [232, 0], [5, 0], [1202, 0], [5, 1202], [0, 1202]]


ತರಬೇತಿ ಡೇಟಾಸೆಟ್ ಅನ್ನು ಸಿದ್ಧಪಡಿಸೋಣ. ನಾವು ಎಲ್ಲಾ ಸುದ್ದಿಗಳನ್ನು ಪರಿಶೀಲಿಸಿ, ಪದ ಜೋಡಿಗಳ ಪಟ್ಟಿಯನ್ನು ಪಡೆಯಲು `to_cbow` ಅನ್ನು ಕರೆಮಾಡುತ್ತೇವೆ, ಮತ್ತು ಆ ಜೋಡಿಗಳನ್ನು `X` ಮತ್ತು `Y` ಗೆ ಸೇರಿಸುತ್ತೇವೆ. ಸಮಯದ ಕಾರಣಕ್ಕಾಗಿ, ನಾವು ಮೊದಲ 10,000 ಸುದ್ದಿಗಳನ್ನು ಮಾತ್ರ ಪರಿಗಣಿಸುವೆವು - ನೀವು ಹೆಚ್ಚು ಸಮಯ ಕಾಯಲು ಸಾಧ್ಯವಿದ್ದರೆ ಮತ್ತು ಉತ್ತಮ ಎಂಬೆಡ್ಡಿಂಗ್‌ಗಳನ್ನು ಪಡೆಯಲು ಬಯಸಿದರೆ, ಈ ಮಿತಿ ಸುಲಭವಾಗಿ ತೆಗೆದುಹಾಕಬಹುದು :)


In [ ]:
X = []
Y = []
for i, x in zip(range(10000), train_dataset):
    for w1, w2 in to_cbow(encode(x[1], vocab), window_size = 5):
        X.append(w1)
        Y.append(w2)

X = torch.tensor(X)
Y = torch.tensor(Y)

ನಾವು ಆ ಡೇಟಾವನ್ನು ಒಂದೇ ಡೇಟಾಸೆಟ್‌ಗೆ ಪರಿವರ್ತಿಸಿ, ಡೇಟಾಲೋಡರ್ ಅನ್ನು ರಚಿಸುವೆವು:


In [ ]:
class SimpleIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, X, Y):
        super(SimpleIterableDataset).__init__()
        self.data = []
        for i in range(len(X)):
            self.data.append( (Y[i], X[i]) )
        random.shuffle(self.data)

    def __iter__(self):
        return iter(self.data)

ನಾವು ಆ ಡೇಟಾವನ್ನು ಒಂದೇ ಡೇಟಾಸೆಟ್‌ಗೆ ಪರಿವರ್ತಿಸಿ, ಡೇಟಾಲೋಡರ್ ಅನ್ನು ರಚಿಸುವೆವು:


In [ ]:
ds = SimpleIterableDataset(X, Y)
dl = torch.utils.data.DataLoader(ds, batch_size = 256)

ಈಗ ನಿಜವಾದ ತರಬೇತಿಯನ್ನು ಮಾಡೋಣ. ನಾವು `SGD` ಆಪ್ಟಿಮೈಜರ್ ಅನ್ನು ತುಂಬಾ ಹೆಚ್ಚಿನ ಲರ್ನಿಂಗ್ ರೇಟ್‌ನೊಂದಿಗೆ ಬಳಸುತ್ತೇವೆ. ನೀವು `Adam` ಮುಂತಾದ ಇತರ ಆಪ್ಟಿಮೈಜರ್‌ಗಳೊಂದಿಗೆ ಕೂಡ ಪ್ರಯೋಗ ಮಾಡಬಹುದು. ನಾವು ಪ್ರಾರಂಭದಲ್ಲಿ 10 ಎಪೋಕ್‌ಗಳಿಗಾಗಿ ತರಬೇತಿ ಮಾಡುತ್ತೇವೆ - ಮತ್ತು ನೀವು ಇನ್ನಷ್ಟು ಕಡಿಮೆ ಲಾಸ್‌ಗಾಗಿ ಈ ಸೆಲ್ ಅನ್ನು ಮರುನಡೆಸಬಹುದು.


In [ ]:
def train_epoch(net, dataloader, lr = 0.01, optimizer = None, loss_fn = torch.nn.CrossEntropyLoss(), epochs = None, report_freq = 1):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr = lr)
    loss_fn = loss_fn.to(device)
    net.train()

    for i in range(epochs):
        total_loss, j = 0, 0, 
        for labels, features in dataloader:
            optimizer.zero_grad()
            features, labels = features.to(device), labels.to(device)
            out = net(features)
            loss = loss_fn(out, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
            j += 1
        if i % report_freq == 0:
            print(f"Epoch: {i+1}: loss={total_loss.item()/j}")

    return total_loss.item()/j

In [ ]:
train_epoch(net = model, dataloader = dl, optimizer = torch.optim.SGD(model.parameters(), lr = 0.1), loss_fn = torch.nn.CrossEntropyLoss(), epochs = 10)

Epoch: 1: loss=5.664632366860172
Epoch: 2: loss=5.632101973960962
Epoch: 3: loss=5.610399051405015
Epoch: 4: loss=5.594621561080262
Epoch: 5: loss=5.582538017415446
Epoch: 6: loss=5.572900234519603
Epoch: 7: loss=5.564951676341915
Epoch: 8: loss=5.558288112064614
Epoch: 9: loss=5.552576955031129
Epoch: 10: loss=5.547634165194347


5.547634165194347

## Word2Vec ಪ್ರಯತ್ನಿಸುವುದು

Word2Vec ಬಳಸಲು, ನಮ್ಮ ಶಬ್ದಕೋಶದಲ್ಲಿರುವ ಎಲ್ಲಾ ಪದಗಳಿಗೆ ಹೊಂದಿಕೊಂಡಿರುವ ವೆಕ್ಟರ್‌ಗಳನ್ನು ಹೊರತೆಗೆಯೋಣ:


In [ ]:
vectors = torch.stack([embedder(torch.tensor(vocab[s])) for s in vocab.itos], 0)

ನೋಡೋಣ, ಉದಾಹರಣೆಗೆ, **ಪ್ಯಾರಿಸ್** ಎಂಬ ಪದವನ್ನು ಹೇಗೆ ವೆಕ್ಟರ್ ಆಗಿ ಎನ್‌ಕೋಡ್ ಮಾಡಲಾಗಿದೆ:


In [ ]:
paris_vec = embedder(torch.tensor(vocab['paris']))
print(paris_vec)

tensor([-0.0915,  2.1224, -0.0281, -0.6819,  1.1219,  0.6458, -1.3704, -1.3314,
        -1.1437,  0.4496,  0.2301, -0.3515, -0.8485,  1.0481,  0.4386, -0.8949,
         0.5644,  1.0939, -2.5096,  3.2949, -0.2601, -0.8640,  0.1421, -0.0804,
        -0.5083, -1.0560,  0.9753, -0.5949, -1.6046,  0.5774],
       grad_fn=<EmbeddingBackward>)


ಪರ್ಯಾಯ ಪದಗಳನ್ನು ಹುಡುಕಲು Word2Vec ಬಳಸುವುದು ಆಸಕ್ತಿದಾಯಕವಾಗಿದೆ. ಕೆಳಗಿನ ಫಂಕ್ಷನ್ ನೀಡಲಾದ ಇನ್‌ಪುಟ್‌ಗೆ `n` ಅತ್ಯಂತ ಸಮೀಪದ ಪದಗಳನ್ನು ಹಿಂತಿರುಗಿಸುತ್ತದೆ. ಅವುಗಳನ್ನು ಕಂಡುಹಿಡಿಯಲು, ನಾವು $|w_i - v|$ ನಾರ್ಮ್ ಅನ್ನು ಲೆಕ್ಕಹಾಕುತ್ತೇವೆ, ಇಲ್ಲಿ $v$ ನಮ್ಮ ಇನ್‌ಪುಟ್ ಪದಕ್ಕೆ ಹೊಂದಿರುವ ವೆಕ್ಟರ್ ಆಗಿದ್ದು, $w_i$ ಶಬ್ದಕೋಶದಲ್ಲಿನ i-ನೇ ಪದದ ಎನ್‌ಕೋಡಿಂಗ್ ಆಗಿದೆ. ನಂತರ ನಾವು ಅರೇ ಅನ್ನು ಸರಿದೂಗಿಸಿ `argsort` ಬಳಸಿ ಸಂಬಂಧಿತ ಸೂಚ್ಯಂಕಗಳನ್ನು ಹಿಂತಿರುಗಿಸುತ್ತೇವೆ, ಮತ್ತು ಮೊದಲ `n` ಅಂಶಗಳನ್ನು ತೆಗೆದುಕೊಳ್ಳುತ್ತೇವೆ, ಅವು ಶಬ್ದಕೋಶದಲ್ಲಿನ ಸಮೀಪದ ಪದಗಳ ಸ್ಥಾನಗಳನ್ನು ಎನ್‌ಕೋಡ್ ಮಾಡುತ್ತವೆ.


In [ ]:
def close_words(x, n = 5):
  vec = embedder(torch.tensor(vocab[x]))
  top5 = np.linalg.norm(vectors.detach().numpy() - vec.detach().numpy(), axis = 1).argsort()[:n]
  return [ vocab.itos[x] for x in top5 ]

close_words('microsoft')

['microsoft', 'quoted', 'lp', 'rate', 'top']

In [ ]:
close_words('basketball')

['basketball', 'lot', 'sinai', 'states', 'healthdaynews']

In [ ]:
close_words('funds')

['funds', 'travel', 'sydney', 'japan', 'business']

## ಸಾರಾಂಶ

CBoW ಮುಂತಾದ ಚತುರ ತಂತ್ರಗಳನ್ನು ಬಳಸಿಕೊಂಡು, ನಾವು Word2Vec ಮಾದರಿಯನ್ನು ತರಬೇತಿಮಾಡಬಹುದು. ಕೇಂದ್ರ ಪದವನ್ನು ನೀಡಿದಾಗ ಹತ್ತಿರದ ಪದವನ್ನು ಊಹಿಸಲು ತರಬೇತಿಮಾಡುವ skip-gram ಮಾದರಿಯನ್ನು ಕೂಡ ಪ್ರಯತ್ನಿಸಿ, ಅದು ಎಷ್ಟು ಚೆನ್ನಾಗಿ ಕಾರ್ಯನಿರ್ವಹಿಸುತ್ತದೆ ಎಂದು ನೋಡಿ.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ಅಸ್ವೀಕರಣ**:  
ಈ ದಸ್ತಾವೇಜು [Co-op Translator](https://github.com/Azure/co-op-translator) ಎಂಬ AI ಅನುವಾದ ಸೇವೆಯನ್ನು ಬಳಸಿ ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ಶುದ್ಧತೆಯತ್ತ ಪ್ರಯತ್ನಿಸುತ್ತಿದ್ದರೂ, ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ತಪ್ಪುಗಳು ಅಥವಾ ಅಸತ್ಯತೆಗಳು ಇರಬಹುದು ಎಂದು ದಯವಿಟ್ಟು ಗಮನಿಸಿ. ಮೂಲ ಭಾಷೆಯಲ್ಲಿರುವ ಮೂಲ ದಸ್ತಾವೇಜನ್ನು ಅಧಿಕೃತ ಮೂಲವೆಂದು ಪರಿಗಣಿಸಬೇಕು. ಮಹತ್ವದ ಮಾಹಿತಿಗಾಗಿ, ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗುತ್ತದೆ. ಈ ಅನುವಾದ ಬಳಕೆಯಿಂದ ಉಂಟಾಗುವ ಯಾವುದೇ ತಪ್ಪು ಅರ್ಥಮಾಡಿಕೊಳ್ಳುವಿಕೆ ಅಥವಾ ತಪ್ಪು ವಿವರಣೆಗಳಿಗೆ ನಾವು ಹೊಣೆಗಾರರಾಗುವುದಿಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
